## Tensorflow

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Reshape
from tensorflow.keras.optimizers import Adam
import pandas as pd
from config import config

from rl.agents import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

from env_mnist import SeqEnvironment
%reload_ext autoreload
%autoreload 2

2023-04-10 13:23:51.914922: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data(path='mnist.npz')
x_train = x_train.reshape(x_train.shape[0],-1) 
x_train.shape, y_train.shape

((60000, 784), (60000,))

In [3]:
data_train = np.concatenate((x_train, y_train.reshape(-1,1)), axis=1)
data_train.shape

(60000, 785)

In [4]:
costs = -1*np.ones(x_train.shape[1])
costs.shape

(784,)

In [5]:
env = SeqEnvironment()

In [16]:
op = env.reset()

In [20]:
op.shape

(1000, 2, 784)

In [17]:
op[0].shape

(2, 784)

In [18]:
op[1].shape

(2, 784)

In [23]:
# def build_model(input_shape, output_shape, NN_SIZE = 512):
#     model = Sequential()
#     model.add(Reshape(target_shape=(-1,input_shape), input_shape=(1,2,784)))
#     model.add(Dense(NN_SIZE, activation='relu'))
#     model.add(Dense(NN_SIZE, activation='relu'))
#     model.add(Dense(NN_SIZE, activation='relu'))
#     model.add(Dense(output_shape, activation='linear'))
#     model.add(Flatten())
#     return model

def build_model(input_shape, output_shape, NN_SIZE = 512):
    model = Sequential()
    model.add(tf.keras.layers.InputLayer((1,1,2,784)))
    model.add(Reshape(target_shape=(-1,input_shape)))
    model.add(Dense(NN_SIZE, activation='relu'))
    model.add(Dense(NN_SIZE, activation='relu'))
    model.add(Dense(NN_SIZE, activation='relu'))
    model.add(Dense(output_shape, activation='linear'))
    model.add(Flatten())
    return model

model = build_model(2*784, 794)
# model.summary()

In [24]:
model.build(input_shape=(None,2,784))

In [25]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_3 (Reshape)         (None, 1, 1568)           0         
                                                                 
 dense_12 (Dense)            (None, 1, 512)            803328    
                                                                 
 dense_13 (Dense)            (None, 1, 512)            262656    
                                                                 
 dense_14 (Dense)            (None, 1, 512)            262656    
                                                                 
 dense_15 (Dense)            (None, 1, 794)            407322    
                                                                 
 flatten (Flatten)           (None, 794)               0         
                                                                 
Total params: 1,735,962
Trainable params: 1,735,962
No

In [26]:
# from keras.layers import Input, Dense
# from keras.models import Model

# def build_model(input_shape, output_shape, NN_SIZE=512):
#     inputs = Input(shape=(input_shape,))
#     x = Dense(NN_SIZE, activation='relu')(inputs)
#     x = Dense(NN_SIZE, activation='relu')(x)
#     x = Dense(NN_SIZE, activation='relu')(x)
#     outputs = Dense(output_shape, activation='linear')(x)
#     model = Model(inputs=inputs, outputs=outputs)
#     return model

# model = build_model(2*784, 794)


In [27]:
# from keras.models import Sequential
# from keras.layers import Dense, Activation, Flatten
# from keras.optimizers import Adam

# model = Sequential()
# model.add(Dense(512, input_shape=(2*config.FEATURE_DIM,)))
# model.add(Activation('relu'))
# model.add(Dense(794))
# model.add(Activation('linear'))
# model.add(Flatten())

# model.summary()

In [28]:
model.output

<tf.Tensor 'flatten/Reshape:0' shape=(None, 794) dtype=float32>

In [29]:
config.ACTION_DIM

794

In [30]:
policy = EpsGreedyQPolicy()
memory = SequentialMemory(limit=50000, window_length=1)
dqn = DQNAgent(model=model, memory=memory, policy=policy, nb_actions=config.ACTION_DIM, nb_steps_warmup=10, target_model_update=1e-2)

In [31]:
# model.fit(np.random.randn())

In [32]:
dqn.compile(tf.keras.optimizers.legacy.Adam(learning_rate=1e-3), metrics=['mae'])

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [ ]:
np.random.choice()

In [33]:
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)


ValueError: Error when checking input: expected input_3 to have shape (1, 1, 2, 784) but got array with shape (1, 1000, 2, 784)

## Torch

In [ ]:
import torch
from torch import nn

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, FEATURE_DIM = 784, NN_SIZE = 512, N_CLASSES = 10, NN_HIDDEN_LAYERS = 3, ACTION_DIM = 10):
        super(Net, self).__init__()

        in_nn  = FEATURE_DIM * 2
        out_nn = NN_SIZE

        self.layers = []
        for i in range(NN_HIDDEN_LAYERS):
            layer = torch.nn.Linear(in_nn, out_nn)
            in_nn = out_nn

            self.layers.append(layer)
            self.add_module("layer "+str(i), layer)

        # dueling architecture
        self.layer_class = torch.nn.Linear(in_nn, 1)
        self.layer_action = torch.nn.Linear(in_nn, ACTION_DIM)
 
        self.opt = torch.optim.Adam(self.parameters(), lr=config.OPT_LR, weight_decay=config.OPT_L2)

        self.loss_mse   = torch.nn.MSELoss()
        self.loss_cross = torch.nn.CrossEntropyLoss()

        self.cuda()

    def forward(self, batch):
        flow = batch.view(-1, self.FEATURE_DIM * 2) # reshape

        for layer in self.layers:
            flow = F.relu(layer(flow))

        v = self.layer_class(flow)
        a = self.layer_action(flow)

        q = v + a - a.mean(dim=1, keepdim=True)
        return q